Last edited by Nanne on 08/01/2024.
## Rush Hour
This is a simple setup for the implementation of Rush Hour.
The code was written using Python 3.9.18.

In [1]:
import numpy as np
import pandas as pd
import os

#### Classes

In [2]:
class Vehicle:
    def __init__(self, length, orientation, position, name):
        self.length = int(length)
        self.orientation = orientation
        self.position = position
        self.name = name

class Board:
    def __init__(self, size):
        self.grid = np.array([[' '] * size] * size, dtype=object)
        self.vehicles = []

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)
        col, row = vehicle.position
        if vehicle.orientation == 'H':
            for i in range(vehicle.length):
                self.grid[row][col + i] = vehicle.name
        else:
            for i in range(vehicle.length):
                self.grid[row + i][col] = vehicle.name


In [3]:
# Saves the boards in a list. 
gameboards = []
path = os.path.join(os.getcwd(), 'gameboards')
for filename in os.listdir(path):
    board_df = pd.read_csv(os.path.join(path, filename))
    board_df.set_index('car', inplace=True)
    gameboards.append(board_df)


Function to set up the board:

In [4]:
def setup_board(gameboard, board_size=6):
    # Set the grid of the board
    board = Board(board_size)
    for index, car in gameboard.iterrows():
        # Set length to be an int instead of string
        length = int(car['length'])

        # Subtract 1 from position to make it zero-indexed
        car['row'] -= 1
        car['col'] -= 1

        # Create vehicle object and add it to the board, index serves as name
        vehicle = Vehicle(length, car['orientation'], (car['col'], car['row']), index)
        board.add_vehicle(vehicle)
    return board

Board printer function:

In [5]:
def print_board(board):
    """ 
    Prints the board in a neat format.
    """
    for row in board.grid:
        print(' '.join(str(x).ljust(2) for x in row))

# Usage:
board = setup_board(gameboards[0])
print_board(board)

   A  A  B  B  B 
   C  C  E  D  D 
X  X  G  E     I 
F  F  G  H  H  I 
K     L     J  J 
K     L          


In [6]:
# Just to show that the boards are saved correctly
for board in gameboards:
    display(board)

,orientation,col,row,length
car,,,,
A,H,2,1,2
B,H,4,1,3
C,H,2,2,2
D,H,5,2,2
E,V,4,2,2
F,H,1,4,2
G,V,3,3,2
H,H,4,4,2
I,V,6,3,2


,orientation,col,row,length
car,,,,
A,H,3,1,2
B,H,5,1,2
C,H,2,2,2
D,H,4,2,2
E,H,1,4,2
F,H,3,4,2
G,V,5,3,2
H,V,6,2,3
I,H,5,5,2


,orientation,col,row,length
car,,,,
A,H,4,1,2
B,V,3,1,3
C,V,6,1,3
D,H,5,4,2
E,H,2,5,2
F,V,1,5,2
G,V,4,4,3
H,H,5,6,2
X,H,4,3,2
